# 二级词包打标

In [5]:
import json

input_file = 'data/JAN_samples_isFAQ_with_1st_label.txt'
output_file = 'data/JAN_samples_isFAQ_with_1st_label_2nd_label_0322.txt'

#加载文件
PH01_dicts = []
with open(input_file,'r',encoding='utf-8') as inf:
    for line in inf:
        load_dict = json.loads(line.strip())
        load_dict["query"] = load_dict.pop("content")
        load_dict["session_id"] = load_dict.pop("sessionid")
        PH01_dicts.append(load_dict)

print(len(PH01_dicts))

label_frame = []

#退保类目打标
label_1_dict = {
 '退保异常': '退保',
 '退保金额异议': '退保',
 '退保咨询': '退保',
 '退保状态查询': '退保',
 '退保成功有短信提醒吗': '退保',
 '退保退费时效': '退保',
 '退保需要什么资料': '退保',
 '退保退费咨询': '退保',
 '退保继续扣费质疑': '退保',
 '购买重复要求退保': '退保',
 '不符合健告退保咨询': '退保',
 '拒赔后要求退保': '退保',
 '退保条件咨询': '退保'
}

label1 = list(set(label_1_dict.values()))
label_dict = {}
for k, v in label_1_dict.items():
    if v not in label_dict:
        label_dict[v] = [k]
    else:
        label_dict[v].append(k)

keywords = {
    '退保异常': {'must': [], 'must_or': ['撤单', '推保', '退', '取消', '不想', '不要'],
             'should': ['没法', '无法', '退不了', '失败', '异常', '闪退', '出错']},

    '退保金额异议': {'must': [], 'must_or': ['撤单', '推保', '退', '取消', '不想', '不要'],
               'should': ['几元', '几块', '这么点', '0元']},

    '退保咨询': {'must': [], 'must_or': ['撤单', '推保', '退', '取消', '不想', '不要'],
             'should': ['怎么退', '操作', '怎么取消', '如何退', '如何取消', '帮我', '给我', '怎么办', '流程']},

    '退保状态查询': {'must': [], 'must_or': ['撤单', '推保', '退', '取消', '不想', '不要'],
               'should': ['退保成功', '退成功', '退完', '申请成功', '成功了', '进度']},

    '退保成功有短信提醒吗': {'must': [], 'must_or': ['撤单', '推保', '退', '取消', '不想', '不要'],
                   'should': ['短信', '消息', '讯息', '提示', '提醒', '收到', '通知']},
    # 应包含保费相关，但是三级的must must_or should逻辑无法包括，是否需要往下延伸
    '退保退费时效': {'must': [], 'must_or': ['撤单', '推保', '退', '取消', '不想', '不要'],
               'should': ['还未', '还没', '多久', '多长', '时间', '时候']},

    '退保需要什么资料': {'must': [], 'must_or': ['撤单', '推保', '退', '取消', '不想', '不要', '解除'],
                 'should': ['提供', '资料', '证件', '身份', '材料', '证明', '照片']},
    # 应包含保费相关
    # 退到哪里 没有包含
    # 是否与退保金额异议重复 —— 钱 元 额 块
    '退保退费咨询': {'must': [], 'must_or': ['撤单', '推保', '退', '取消', '不想', '不要'],
               'should': ['多少', '多少钱', '计算', '退回', '全款', '退钱', '退保费',
                          '百分之', '还钱', '本钱', '返还', '退还', '钱可以', '损失', '钱']},

    '退保继续扣费质疑': {'must': [], 'must_or': ['撤单', '推保', '退', '取消', '不想', '不要', '没有买', '中止', '终止'],
                 'should': ['还扣', '还在扣', '继续扣', '自动扣', '又扣', '再扣']},

    '购买重复要求退保': {'must': [], 'must_or': ['撤单', '推保', '退', '取消', '不想', '不要'],
                 'should': ['买了', '全民保', '重疾险', '买重', '重复', '买多']},

    '不符合健告退保咨询': {'must': [], 'must_or': ['撤单', '推保', '退', '取消', '不想', '不要'],
                  'should': ['健告', '健康告知', '身体', '健康']},

    '拒赔后要求退保': {'must': [], 'must_or': ['撤单', '推保', '退', '取消', '不想', '不要'],
                'should': ['拒赔', '不赔', '不能赔', '理赔', '报销', '不能报销', '无法报销', '赔不了', '报不了']},

    '退保条件咨询': {'must': [], 'must_or': ['撤单', '推保', '退', '取消', '不想', '不要', "终止", "中止"],
               'should': ['随时', '可以退', '能不能', '可以退吗']}
}

keyword_dict = {}
for item in keywords.keys():
    keyword_dict[item] = 0

surrender_match_count = 0
surrender_unmatch_count = 0
for dic in PH01_dicts:
    if dic['label_1st'] == "退保":
        words = dic['query']
        match_flag = False

        for k, v in keywords.items():
            must_flag = True
            for item in v['must']:
                if item not in words:
                    must_flag = False
                    break
            if must_flag:
                must_or_flag = False
                if len(v['must_or']) == 0:
                    must_or_flag = True
                else:
                    for item in v['must_or']:
                        if item in words:
                            must_or_flag = True
                            break
            if must_flag and must_or_flag:
                should_flag = False
                if len(v['should']) == 0:
                    should_flag = True
                else:
                    for item in v['should']:
                        if item in words:
                            should_flag = True
                            break
            if must_flag and must_or_flag and should_flag:
                keyword_dict[k] += 1
                match_flag = True
                dic['label_2nd'] = k

        if not match_flag:
            surrender_unmatch_count += 1
            dic['label_2nd'] = "表达退保意愿"

        if match_flag:
            surrender_match_count += 1

surrender_cover_rate = round(surrender_match_count / (surrender_match_count + surrender_unmatch_count), 4) * 100
print("退保类目二级标签覆盖率为：" + str(surrender_cover_rate) + "%")

label_frame.append(["退保", surrender_match_count, surrender_cover_rate, 
                    round(surrender_match_count / len(PH01_dicts), 4) * 100])

#续保
label_1_dict = {
    '续保年限咨询': '续保',
    '续保条件咨询': '续保',
    '产品停售续保咨询': '续保',
    '生病续保咨询': '续保',
    '出险续保咨询': '续保',
    '续保健告&等待期咨询': '续保',
    '续保扣款方式咨询': '续保',
    '续保扣费异议': '续保',
    '续保升级咨询': '续保',
    '续保状态咨询': '续保',
    '续保保费金额异议': '续保',
    '续保保费咨询': '续保',
    '续保保费咨询（>超过65岁)': '续保',
    '续保流程咨询': '续保',
    '续保折扣咨询': '续保',
    '续保增值服务': '续保',
    '续保异常': '续保',
    '续保手机号修改': '续保',
    '关闭续保后扣费问题': '续保',
    '关闭自动续保': '续保',
    '到期关闭自动续保': '续保',
    '保费上涨关闭续保': '续保',
    '关闭续保退费咨询': '续保'
}

label1 = list(set(label_1_dict.values()))
label_dict = {}
for k, v in label_1_dict.items():
    if v not in label_dict:
        label_dict[v] = [k]
    else:
        label_dict[v].append(k)

keywords = {
    '续保年限咨询': {'must': ['续'], 'must_or': [],
               'should': ['年限', '几年', '终身', '年龄', '几岁', '多少岁', '长期',
                          '年限', '60岁', '70岁', '80岁', '90岁', '100岁', '超龄']},

    '续保条件咨询': {'must': ['续'], 'must_or': [],
               'should': ['条件', '保证', '年年', '6年', '六年', '可以再', '一直', '二十年', '20年']},

    '产品停售续保咨询': {'must': ['续'], 'must_or': [], 'should': ['停售', '下架', '不售']},

    '生病续保咨询': {'must': ['续'], 'must_or': [],
               'should': ['血', '胃', '肾', '三高', '症', '癌', '患', '瘤', '病', '肝', '住院', '身故', '确诊',
                          '脂', '症', '住院', '哪些', '肺', '风湿', '炎', '痛', '病', '心', '腺', '得了']},
    # 出险合同终止->续保的反义词
    '出险续保咨询': {'must': [], 'must_or': ['续', '合同', '中止', '终止'],
               'should': ['赔', '出险', '出过险', '报']},

    '续保健告&等待期咨询': {'must': ['续'], 'must_or': [],
                   'should': ['过渡期', '免赔期', '观察期', '等待期', '告知', '健康告知', '健告']},

    '续保扣款方式咨询': {'must': ['扣'], 'must_or': ['如何', '改', '更正', '卡', '方式', '绑定', '余额'], 'should': []},

    '续保扣费异议': {'must': ['扣'], 'must_or': ['续'], 'should': ['费', '钱', '额', '元']},

    '续保升级咨询': {'must': ['续'], 'must_or': [], 'should': ['升级']},

    '续保状态咨询': {'must': ['续'], 'must_or': [], 'should': ['成功', '自动', '开通'],
               'none': ['取消', '关闭', '不自动续保', '不要开通', '不开通', '未开通']},

    '续保保费金额异议': {'must': [], 'must_or': ['续', '扣'],
                 'should': ['贵', '涨', '长了', '高', '增', '升', '上调', '翻倍', '浮动', '上浮']},

    '续保保费咨询': {'must': ['续'], 'must_or': [], 'should': ['保费', '价格', '加收', '加钱', '元', '钱', '费']},

    '续保保费咨询（>超过65岁)': {'must': ['续'], 'must_or': [], 'should': ['65', '六十五', '66', '六十六']},
    # 未完成
    '续保流程咨询': {'must': ['续'], 'must_or': [], 'should': ['怎么', '如何', '操作', '可以', '没法']},

    '续保折扣咨询': {'must': ['续'], 'must_or': [], 'should': ['打折', '折扣', '几折', '优惠', '递减', ]},

    '续保增值服务': {'must': ['续'], 'must_or': [], 'should': ['领', '福利', '增值服务', '送', '赠']},

    '续保异常': {'must': ['续'], 'must_or': [], 'should': ['不可以', '无法', '不能', '续不了']},

    '续保手机号修改': {'must': ['续'], 'must_or': [], 'should': ['手机号', '号码']},

    '关闭续保后扣费问题': {'must': [], 'must_or': ['续', '关闭', '取消', '停', '同意'], 'should': ['扣', '私自']},

    '关闭自动续保': {'must': [], 'must_or': ['续', '扣'], 'should': ['不续', '不用续', '关闭', '取消', '停', '同意']},

    '到期关闭自动续保': {'must': [], 'must_or': ['续'], 'should': ['到期', '过期']},

    '保费上涨关闭续保': {'must': ['续'], 'must_or': ['涨', '高', '贵', '多'], 'should': ['退', '取消', '停']},

    '关闭续保退费咨询': {'must': ['续'], 'must_or': ['钱', '费'], 'should': ['可以退', '如何']}
}

keyword_dict = {}
for item in keywords.keys():
    keyword_dict[item] = 0
renewal_match_count = 0
renewal_unmatch_count = 0

for dic in PH01_dicts:
    if dic['label_1st'] == "续保":
        words = dic['query']
        match_flag = False

        for k, v in keywords.items():
            must_flag = True
            for item in v['must']:
                if item not in words:
                    must_flag = False
                    break
            if must_flag:
                must_or_flag = False
                if len(v['must_or']) == 0:
                    must_or_flag = True
                else:
                    for item in v['must_or']:
                        if item in words:
                            must_or_flag = True
                            break
            if must_flag and must_or_flag:
                should_flag = False
                if len(v['should']) == 0:
                    should_flag = True
                else:
                    for item in v['should']:
                        if item in words:
                            should_flag = True
                            break
            if must_flag and must_or_flag and should_flag:
                keyword_dict[k] += 1
                match_flag = True
                dic['label_2nd'] = k

        if not match_flag:
            renewal_unmatch_count += 1
            dic['label_2nd'] = "咨询续保问题"

        if match_flag:
            renewal_match_count += 1

renewal_cover_rate = round(renewal_match_count / (renewal_match_count + renewal_unmatch_count), 4) * 100
print("续保类目二级标签覆盖率为：" + str(renewal_cover_rate) + "%")

label_frame.append(["续保", renewal_match_count, renewal_cover_rate, 
                    round(renewal_match_count / len(PH01_dicts), 4) * 100])

#理赔
label_1_dict = {
    '咨询理赔范围': '理赔',
    '咨询理赔资料': '理赔',
    '咨询理赔流程': '理赔',
    '咨询理赔金额': '理赔',
    '咨询理赔进度/时效': '理赔',
    '咨询赔付比例': '理赔',
    '理赔结论(拒赔）不满': '理赔',
    '咨询押金垫付': '理赔',
    '押金垫付流程': '理赔',
    '押金垫付材料': '理赔',
    '押金垫付金额': '理赔',
    '押金垫付条件': '理赔'
}

label1 = list(set(label_1_dict.values()))
label_dict = {}
for k, v in label_1_dict.items():
    if v not in label_dict:
        label_dict[v] = [k]
    else:
        label_dict[v].append(k)

keywords = {
    '咨询理赔范围': {'must': [], 'must_or': ["赔", "报"],
               'should': ['可以赔', '可以理赔', '可以报',
                          '影响', '情况', '情形'
                                      '血', '胃', '肾', '三高', '重症', '癌', '患', '瘤', '病', '肝', '脂', '症',
                          '轻症', '重症', '身故',
                          '范围', '包括', '哪些', '肺', '风湿', '炎', '痛', '病', '牙',
                          '车祸', '抓伤', '咬伤', '骨折', '摔', '扭', '臂', '胳膊',
                          '确诊', '住院', '住过院', '医院', '门诊', '手术']},

    '咨询理赔资料': {'must': [], 'must_or': ["赔", "报"],
               'should': ['材料', '资料', '递', '寄', '邮',
                          '纸质', '复印件', '原件', '单据',
                          '发票', '身份证', '病理报告', '检查报告', '病历',
                          '申请书']},

    '咨询理赔流程': {'must': [], 'must_or': ["赔", "报"],
               'should': ['怎么', '如何', '操作', '渠道', '申请', '流程', '步骤', '指引', '勾选', '手续']},

    '咨询理赔金额': {'must': [], 'must_or': ['赔', '报'],
               'should': ['多少', '金额', '钱', '元', '块',
                          '几次', '多少词']},

    '咨询理赔进度/时效': {'must': [], 'must_or': ['赔', '报'],
                  'should': ['进度', '进展', '什么情况', '啥情况', '提交',
                             '多久', '多长时间', '什么时', '这么久', '到账']},

    '咨询赔付比例': {'must': [], 'must_or': ['赔', '报'],
               'should': ['比例', '全部', '部分', '100%', '百分之百', '百分百']},

    '理赔结论(拒赔）不满': {'must': [], 'must_or': ['赔', '报'],
                   'should': ['拒绝', '拒赔', '凭什么', '扯皮']},

    '押金垫付相关': {'must': [], 'must_or': ["垫", "押金"], 'should': []}

}

keyword_dict = {}
for item in keywords.keys():
    keyword_dict[item] = 0

settle_match_count = 0
settle_unmatch_count = 0

for dic in PH01_dicts:
    if dic['label_1st'] == "理赔":
        words = dic['query']
        match_flag = False

        for k, v in keywords.items():
            must_flag = True
            for item in v['must']:
                if item not in words:
                    must_flag = False
                    break
            if must_flag:
                must_or_flag = False
                if len(v['must_or']) == 0:
                    must_or_flag = True
                else:
                    for item in v['must_or']:
                        if item in words:
                            must_or_flag = True
                            break
            if must_flag and must_or_flag:
                should_flag = False
                if len(v['should']) == 0:
                    should_flag = True
                else:
                    for item in v['should']:
                        if item in words:
                            should_flag = True
                            break
            if must_flag and must_or_flag and should_flag:
                keyword_dict[k] += 1
                match_flag = True
                dic['label_2nd'] = k

        if match_flag == False:
            settle_unmatch_count += 1
            dic['label_2nd'] = "咨询理赔问题"

        if match_flag:
            settle_match_count += 1

settle_cover_rate = round(settle_match_count / (settle_match_count + settle_unmatch_count), 4) * 100
print("理赔类目二级标签覆盖率为：" + str(settle_cover_rate) + "%")

label_frame.append(["理赔", settle_match_count, settle_cover_rate, 
                    round(settle_match_count / len(PH01_dicts), 4) * 100])

#保费相关
label_1_dict = {
    '具体保费咨询': '保费',
    '保费金额变化咨询': '保费',
    '咨询保费缴纳方式': '保费',
    '咨询保费金额': '保费',
    '保费问题咨询': '保费',
    '咨询保额问题': '保费',
    '咨询费率表': '保费',
    '保费差异异议': '保费'
}

label1 = list(set(label_1_dict.values()))
label_dict = {}
for k, v in label_1_dict.items():
    if v not in label_dict:
        label_dict[v] = [k]
    else:
        label_dict[v].append(k)

keywords = {
    '具体保费咨询': {'must': ["保费"], 'must_or': ["是", "多少", "去年", "上年", "年龄", "年纪", "岁数"], 'should': []},

    '保费差异异议': {'must': [],
                'must_or': ["保费", "费率"],
                'should': ["配偶", "爱人", "不一样", "差异", "不同", "区别", "相差", "差别", "一样", "多", "实收", "实际", "高"]},

    '保费金额变化咨询': {'must': ["保费"],
                 'must_or': ["以后", "涨", "高", "增加", "变贵", "变化", "递增", "浮动", "随着", "上升", "升高",
                             "增", "上调",
                             "比去年", "逐年", "每年", "每一年", "一年", "一个月", "年年", "今年", "每月",
                             "贵"
                             ],
                 'should': []},

    '咨询保费金额': {'must': ["保费"], 'must_or': ["月", "年"], 'should': []},

    '咨询保费缴纳方式': {'must': [],
                 'must_or': ['年缴', '月缴', '连缴', '付费', '一缴', '一交', '年交', '月交',
                             '交一年', '一次性', '按月', '分月', '月支付', '年支付',
                             '月付', '年付', '到期'],
                 'should': []},
    '保费问题咨询': {'must': ["保费"], 'must_or': ["自己交", "帮", "替"], 'should': []},

    '咨询保额问题': {'must': [], 'must_or': ["保额", "额度"], 'should': []},

    '咨询费率表': {'must': [], 'must_or': ["费率", "保费表"], 'should': ["年龄", "年纪"]},

}

keyword_dict = {}
for item in keywords.keys():
    keyword_dict[item] = 0

fee_match_count = 0
fee_unmatch_count = 0
for dic in PH01_dicts:
    if dic['label_1st'] == "保费":
        words = dic['query']
        match_flag = False

        for k, v in keywords.items():
            must_flag = True
            for item in v['must']:
                if item not in words:
                    must_flag = False
                    break
            if must_flag:
                must_or_flag = False
                if len(v['must_or']) == 0:
                    must_or_flag = True
                else:
                    for item in v['must_or']:
                        if item in words:
                            must_or_flag = True
                            break
            if must_flag and must_or_flag:
                should_flag = False
                if len(v['should']) == 0:
                    should_flag = True
                else:
                    for item in v['should']:
                        if item in words:
                            should_flag = True
                            break
            if must_flag and must_or_flag and should_flag:
                keyword_dict[k] += 1
                match_flag = True
                dic['label_2nd'] = k

        if match_flag == False:
            fee_unmatch_count += 1
            dic['label_2nd'] = "保费问题咨询"

        if match_flag:
            fee_match_count += 1

fee_cover_rate = round(fee_match_count / (fee_match_count + fee_unmatch_count), 4) * 100

label_frame.append(["保费", fee_match_count, fee_cover_rate, 
                    round(fee_match_count / len(PH01_dicts), 4) * 100])

#保单问题咨询
label_1_dict = {

    '保单信息查询': '保单发票',
    '保单页操作问题': '保单发票',
    '保单问题咨询': '保单发票',
    '发票问题咨询': '保单发票',
    '电子保单获取流程': '保单发票',
    '保单状态查询': '保单发票',
    '保单获取': '保单发票',
    '纸质保单配送服务': '保单发票'
}

label1 = list(set(label_1_dict.values()))
label_dict = {}
for k, v in label_1_dict.items():
    if v not in label_dict:
        label_dict[v] = [k]
    else:
        label_dict[v].append(k)

keywords = {

    '保单信息查询': {'must': [], 'must_or': ["保单", "发票", "合同"],
               'should': ["没", "查", "看", "找", "哪", "收", "发", "详情", "不知道"]},

    '保单页操作问题': {'must': [], 'must_or': ["打不开", "进不去", "链接", "页"], 'should': []},

    '保单问题咨询': {'must': ["保单"], 'must_or': [], 'should': ["份", "张", "个", "不一样", "意思", "看不到", "查不到", "不清楚"]},

    '发票问题咨询': {'must': [], 'must_or': ["发票", "开票"], 'should': ["改", "开"]},

    '电子保单获取流程': {'must': ["保单"], 'must_or': ["下载", "打", "印", "邮箱"], 'should': []},

    '保单状态查询': {'must': ["保单"],
               'must_or': ["生效", "生成", "失效", "到期", "出来", "过期", "正常", "暂无", "下来", "保障中", "承保中", "承包中"],
               'should': []},

    '保单获取': {'must': ["保单"], 'must_or': ["电子", "纸"], 'should': []},

    '纸质保单配送服务': {'must': ["保单"], 'must_or': ["邮", "寄", "到"], 'should': []},


}

keyword_dict = {}
for item in keywords.keys():
    keyword_dict[item] = 0

policy_match_count = 0
policy_unmatch_count = 0

for dic in PH01_dicts:
    if dic['label_1st'] == "保单发票":
        words = dic['query']
        match_flag = False

        for k, v in keywords.items():
            must_flag = True
            for item in v['must']:
                if item not in words:
                    must_flag = False
                    break
            if must_flag:
                must_or_flag = False
                if len(v['must_or']) == 0:
                    must_or_flag = True
                else:
                    for item in v['must_or']:
                        if item in words:
                            must_or_flag = True
                            break
            if must_flag and must_or_flag:
                should_flag = False
                if len(v['should']) == 0:
                    should_flag = True
                else:
                    for item in v['should']:
                        if item in words:
                            should_flag = True
                            break
            if must_flag and must_or_flag and should_flag:
                keyword_dict[k] += 1
                match_flag = True
                dic['label_2nd'] = k

        if match_flag == False:
            policy_unmatch_count += 1
            dic['label_2nd'] = "保单问题咨询"

        if match_flag:
            policy_match_count += 1
            
policy_cover_rate = round(policy_match_count / (policy_match_count + policy_unmatch_count), 4) * 100

label_frame.append(["保单发票", policy_match_count, policy_cover_rate, 
                    round(policy_match_count / len(PH01_dicts), 4) * 100])

#实名认证
label_1_dict = {

    '实名认证操作': '实名认证',
    '实名认证失败': '实名认证',
    '清除实名认证': '实名认证',
    '不想进行实名认证': '实名认证',
    '实名认证问题咨询': '实名认证'}

label1 = list(set(label_1_dict.values()))
label_dict = {}
for k, v in label_1_dict.items():
    if v not in label_dict:
        label_dict[v] = [k]
    else:
        label_dict[v].append(k)

keywords = keyword = {
    # 一级标签本来即与实名相关
    '实名认证问题咨询': {'must': [], 'must_or': ["实名", "认证", "保单"], 'should': []},

    '实名认证操作': {'must': [], 'must_or': ["实名", "认证"],
               'should': ["怎么", "哪", "需要", "操作", "什么是", "没有实名",
                          "身边", "身旁", "刷脸", "人脸", "面部",
                          "名字", "姓名", "要求"]},

    '实名认证失败': {'must': [], 'must_or': ["实名", "认证"],
               'should': ["非本人", "失败", "错误", "不过", "不成功", "不能", "不对", "不给",
                          "没办法", "没法", "无法", "不过", "不了", "通过", "没办法"]},

    '清除实名认证': {'must': [], 'must_or': ["实名", "认证"], 'should': ["取消", "注销", "删除", "清除", "解绑", "撤销", "解除", "消除"
        , "清空"]},

    '不想进行实名认证': {'must': [], 'must_or': ["实名", "认证"], 'should': ["不实名", "不认证", "不想", "不要", "退出"]}
}

keyword_dict = {}
for item in keywords.keys():
    keyword_dict[item] = 0

realname_match_count = 0
realname_unmatch_count = 0

for dic in PH01_dicts:
    if dic['label_1st'] == "实名认证":
        words = dic['query']
        match_flag = False

        for k, v in keywords.items():
            must_flag = True
            for item in v['must']:
                if item not in words:
                    must_flag = False
                    break
            if must_flag:
                must_or_flag = False
                if len(v['must_or']) == 0:
                    must_or_flag = True
                else:
                    for item in v['must_or']:
                        if item in words:
                            must_or_flag = True
                            break
            if must_flag and must_or_flag:
                should_flag = False
                if len(v['should']) == 0:
                    should_flag = True
                else:
                    for item in v['should']:
                        if item in words:
                            should_flag = True
                            break
            if must_flag and must_or_flag and should_flag:
                keyword_dict[k] += 1
                match_flag = True
                dic['label_2nd'] = k

        if match_flag == False:
            realname_unmatch_count += 1
            dic['label_2nd'] = "实名认证问题咨询"

        if match_flag:
            realname_match_count += 1
            
realname_cover_rate = round(realname_match_count / (realname_match_count + realname_unmatch_count), 4) * 100
            
label_frame.append(["实名认证", realname_match_count, realname_cover_rate, 
                    round(realname_match_count / len(PH01_dicts), 4) * 100])

#投诉
label_1_dict = {

    '扣费投诉': '投诉',
    '退保投诉': '投诉',
    '客服投诉': '投诉',
    '保费投诉': '投诉',
    '表达投诉意愿': '投诉',
    '对外投诉': '投诉',
    '投诉渠道咨询': '投诉',
    '信息安全投诉': '投诉',
    '平台投诉': '投诉',
    '保司投诉': '投诉'
}

label1 = list(set(label_1_dict.values()))
label_dict = {}
for k, v in label_1_dict.items():
    if v not in label_dict:
        label_dict[v] = [k]
    else:
        label_dict[v].append(k)

keywords = {

    '扣费投诉': {'must': [],
             'must_or': ["扣"],
             'should': []},

    '退保投诉': {'must': [], 'must_or': ["没入保", "不参保", "退", "取消", "停", "撤", "还给", "还钱", "收不到钱", "退完", "全额"],
             'should': []},

    '客服投诉': {'must': [], 'must_or': ["售后", "服务", "客服", "相关人员", "工号", "工作人员", "管家", "顾问", "服务态度"], 'should': []},

    '保费投诉': {'must': [], 'must_or': ["每月", "首月", "次月", "第一个月", "第二个月", "2个月"], 'should': []},

    '表达投诉意愿': {'must': [], 'must_or': ["骗", "诈", "广告", "虚假", "投诉"], 'should': []},

    '对外投诉': {'must': [],
             'must_or': ["警", "法院", "局", "部", "监", "委", "协", "315", "媒体", "记者"],
             'should': []},

    '投诉渠道咨询': {'must': [], 'must_or': ["投诉电话", "投诉号码", "投诉链接", "如何投诉", "怎么投诉", "哪里", "渠道", "咋"], 'should': []},

    '信息安全投诉': {'must': [], 'must_or': ["身份", "信息", "泄露", "隐私", "外泄"], 'should': []},

    '平台投诉': {'must': [], 'must_or': ["平台", "公司", "保单", "保费表", "费率表", "骚扰"], 'should': []},

    '保司投诉': {'must': [], 'must_or': ["泰康"], 'should': []}

}

keyword_dict = {}
for item in keywords.keys():
    keyword_dict[item] = 0

complaint_match_count = 0
complaint_unmatch_count = 0

for dic in PH01_dicts:
    if dic['label_1st'] == "投诉":
        words = dic['query']
        match_flag = False

        for k, v in keywords.items():
            must_flag = True
            for item in v['must']:
                if item not in words:
                    must_flag = False
                    break
            if must_flag:
                must_or_flag = False
                if len(v['must_or']) == 0:
                    must_or_flag = True
                else:
                    for item in v['must_or']:
                        if item in words:
                            must_or_flag = True
                            break
            if must_flag and must_or_flag:
                should_flag = False
                if len(v['should']) == 0:
                    should_flag = True
                else:
                    for item in v['should']:
                        if item in words:
                            should_flag = True
                            break
            if must_flag and must_or_flag and should_flag:
                keyword_dict[k] += 1
                match_flag = True
                dic['label_2nd'] = k

        if match_flag == False:
            complaint_unmatch_count += 1
            dic['label_2nd'] = "表达投诉意愿"

        if match_flag:
            complaint_match_count += 1
            
complaint_cover_rate = round(complaint_match_count / (complaint_match_count + complaint_unmatch_count), 4) * 100
            
label_frame.append(["投诉", complaint_match_count, complaint_cover_rate, 
                    round(complaint_match_count / len(PH01_dicts), 4) * 100])

#扣费
label_1_dict = {

    '取消自动扣费': '扣费',
    '要求扣费退回': '扣费',
    '扣费要求退保': '扣费',
    '退保继续扣费质疑': '扣费',
    '重复扣费异议': '扣费',
    '关闭续保后扣费问题': '扣费',
    '扣费投诉': '扣费',
    '扣费问题咨询': '扣费',
    '月扣费疑问': '扣费',
    '扣费路径疑问': '扣费'
}

label1 = list(set(label_1_dict.values()))
label_dict = {}
for k, v in label_1_dict.items():
    if v not in label_dict:
        label_dict[v] = [k]
    else:
        label_dict[v].append(k)

keywords = {

    '扣费要求退保': {'must': ["扣"], 'must_or': ["不想", "不再", "退了", "退保"], 'should': []},
    '要求扣费退回': {'must': ["扣"], 'must_or': ["退", "还给", "回", "归"], 'should': []},
    '退保继续扣费质疑': {'must': ["扣"], 'must_or': ["还扣", "还在扣", "还要扣", "还给扣", "还给我扣"], 'should': []},
    '重复扣费异议': {'must': [], 'must_or': ["重复", "两次", "两回", "多次", "多扣", "多收", "多回", "两笔", "三次"], 'should': []},
    '关闭续保后扣费问题': {'must': [], 'must_or': ["关闭", "续"], 'should': []},
    '取消自动扣费': {'must': ["自动"], 'must_or': [], 'should': []},
    '扣费投诉': {'must': ["扣"], 'must_or': ["投诉", "警", "法院", "局", "部", "监", "委", "协", "315", "媒体", "记者", "告你"],
             'should': []},
    '月扣费疑问': {'must': ["扣"], 'must_or': ["每月", "每个月", "第二个月", "2个月", "下月", "下个月", "一个月", "1月", "这个月"], 'should': []},
    '扣费路径疑问': {'must': [], 'must_or': ["银行卡", "卡里", "信用卡", "红包", "转帐", "余额", "零钱", "行卡"], 'should': []},
}

keyword_dict = {}
for item in keywords.keys():
    keyword_dict[item] = 0

deduction_match_count = 0
deduction_unmatch_count = 0

for dic in PH01_dicts:
    if dic['label_1st'] == "扣费相关":
        words = dic['query']
        match_flag = False

        for k, v in keywords.items():
            must_flag = True
            for item in v['must']:
                if item not in words:
                    must_flag = False
                    break
            if must_flag:
                must_or_flag = False
                if len(v['must_or']) == 0:
                    must_or_flag = True
                else:
                    for item in v['must_or']:
                        if item in words:
                            must_or_flag = True
                            break
            if must_flag and must_or_flag:
                should_flag = False
                if len(v['should']) == 0:
                    should_flag = True
                else:
                    for item in v['should']:
                        if item in words:
                            should_flag = True
                            break
            if must_flag and must_or_flag and should_flag:
                keyword_dict[k] += 1
                match_flag = True
                dic['label_2nd'] = k

        if match_flag == False:
            deduction_unmatch_count += 1
            dic['label_2nd'] = "扣费问题咨询"

        if match_flag:
            deduction_match_count += 1
            
deduction_cover_rate = round(deduction_match_count / (deduction_match_count + deduction_unmatch_count), 4) * 100
            
label_frame.append(["扣费相关", deduction_match_count, deduction_cover_rate, 
                    round(deduction_match_count / len(PH01_dicts), 4) * 100])

#保单数据修改
label_1_dict = {

    '修改保单联系方式': '保单信息修改',
    '修改投保人和被保人': '保单信息修改',
    '修改保单身份信息': '保单信息修改',
    '修改保单紧急联系人': '保单信息修改',
    '保单信息修改不成功咨询': '保单信息修改'

}

label1 = list(set(label_1_dict.values()))
label_dict = {}
for k, v in label_1_dict.items():
    if v not in label_dict:
        label_dict[v] = [k]
    else:
        label_dict[v].append(k)

keywords = {

    '修改保单联系方式': {'must': [], 'must_or': ["手机", "电话"], 'should': ["改", "变", "换", "留错", "填错", "写错"]},
    '修改投保人和被保人': {'must': [], 'must_or': ["受益人", "投保人", "被保人"], 'should': ["改", "填错", "换", "变", "写错"]},
    '修改保单身份信息': {'must': [], 'must_or': ["身份", "户口", "姓", "名"], 'should': ["改", "填错", "换", "变", "写错"]},
    '修改保单紧急联系人': {'must': ["紧急联系人"], 'must_or': [], 'should': []},
    '保单信息修改不成功咨询': {'must': [], 'must_or': ["原来", "以前", "没改", "没给", "之前", "本来"], 'should': []}
}

keyword_dict = {}
for item in keywords.keys():
    keyword_dict[item] = 0

policy_change_match_count = 0
policy_change_unmatch_count = 0

for dic in PH01_dicts:
    if dic['label_1st'] == "保单信息修改":
        words = dic['query']

        match_flag = False

        for k, v in keywords.items():
            must_flag = True
            for item in v['must']:
                if item not in words:
                    must_flag = False
                    break
            if must_flag:
                must_or_flag = False
                if len(v['must_or']) == 0:
                    must_or_flag = True
                else:
                    for item in v['must_or']:
                        if item in words:
                            must_or_flag = True
                            break
            if must_flag and must_or_flag:
                should_flag = False
                if len(v['should']) == 0:
                    should_flag = True
                else:
                    for item in v['should']:
                        if item in words:
                            should_flag = True
                            break
            if must_flag and must_or_flag and should_flag:
                keyword_dict[k] += 1
                match_flag = True
                dic['label_2nd'] = k
                mark = dic["query"] + dic["session_id"]

        if match_flag == False:
            policy_change_unmatch_count += 1
            dic['label_2nd'] = "保单信息修改咨询"

        if match_flag:
            policy_change_match_count += 1
            
policy_change_cover_rate = round(policy_change_match_count / (policy_change_match_count + policy_change_unmatch_count), 4) * 100
            
label_frame.append(["保单信息修改", policy_change_match_count, policy_change_cover_rate, 
                    round(policy_change_match_count / len(PH01_dicts), 4) * 100])

#核保
label_1_dict = {

    '咨询体检问题': '核保',
    '咨询健康告知': '核保',
    '咨询免健告': '核保',
    '咨询核保流程': '核保',
    '咨询核保问题': '核保'
}

label1 = list(set(label_1_dict.values()))
label_dict = {}
for k, v in label_1_dict.items():
    if v not in label_dict:
        label_dict[v] = [k]
    else:
        label_dict[v].append(k)

keywords = {

    '咨询体检问题': {'must': ["体检"],
               'must_or': [], 'should': []},
    '咨询健康告知': {'must': [], 'must_or': ["健康告知", "健告"], 'should': []},
    '咨询免健告': {'must': [], 'must_or': ["健康告知", "健告"], 'should': ["免"]},
    '咨询核保流程': {'must': ["核保"], 'must_or': ["怎么", "看", "找", "哪", "弄", "智能", "人工", "医生", "页", "跳", "进行", "操作"],
               'should': []},
    '咨询核保问题': {'must': ["核保"], 'must_or': ["选", "资格", "符合", "影响", "资料", "项", "问", "为什么", "结论", "记录"], 'should': []}

}

keyword_dict = {}
for item in keywords.keys():
    keyword_dict[item] = 0
total = 0
underwriting_match_count = 0
underwriting_unmatch_count = 0

for dic in PH01_dicts:
    if dic['label_1st'] == "核保":
        words = dic['query']
        total += 1
        match_flag = False

        for k, v in keywords.items():
            must_flag = True
            for item in v['must']:
                if item not in words:
                    must_flag = False
                    break
            if must_flag:
                must_or_flag = False
                if len(v['must_or']) == 0:
                    must_or_flag = True
                else:
                    for item in v['must_or']:
                        if item in words:
                            must_or_flag = True
                            break
            if must_flag and must_or_flag:
                should_flag = False
                if len(v['should']) == 0:
                    should_flag = True
                else:
                    for item in v['should']:
                        if item in words:
                            should_flag = True
                            break
            if must_flag and must_or_flag and should_flag:
                keyword_dict[k] += 1
                match_flag = True
                dic['label_2nd'] = k
                mark = dic["query"] + dic["session_id"]

        if match_flag == False:
            underwriting_unmatch_count += 1
            dic['label_2nd'] = "咨询核保问题"

        if match_flag:
            underwriting_match_count += 1
            
underwriting_cover_rate = round(underwriting_match_count / (underwriting_match_count + underwriting_unmatch_count), 4) * 100            
            
label_frame.append(["核保", underwriting_match_count, underwriting_cover_rate, 
                    round(underwriting_match_count / len(PH01_dicts), 4) * 100])

#平台体验
label_1_dict = {
    '页面无法打开': '平台体验',
    '页面展示操作问题': '平台体验',
    '信息展示错误': '平台体验'
}

label1 = list(set(label_1_dict.values()))
label_dict = {}
for k, v in label_1_dict.items():
    if v not in label_dict:
        label_dict[v] = [k]
    else:
        label_dict[v].append(k)

keywords = {
    '页面无法打开': {'must': [], 'must_or': [],
               'should': ["页面", "网址", "白屏", "黑屏", "打开", "打不开", "进不去", "空白", "没反应", "没有进入", "闪退", "升级中", "乱码", "操作频繁",
                          "点不开", "空的"]},

    '页面展示操作问题': {'must': [], 'must_or': [],
                 'should': ["页面", "网址", "打开", "点开", "找不到", "没看", "没找", "没有", "右上角", "操作", "退回", "退出", "显示", "蓝色字体",
                            "跳", "下一步", "选项", "切换", "出现", "按钮", "模块", "界面"]},

    '信息展示错误': {'must': [], 'must_or': ["曾用名", "显示", "之前的", "原来的", "暂无", "没有信息", "不一样", "没有信息", "更换"],
               'should': []}
}

keyword_dict = {}
for item in keywords.keys():
    keyword_dict[item] = 0

platform_match_count = 0
platform_unmatch_count = 0

for dic in PH01_dicts:
    if dic['label_1st'] == "平台体验":
        words = dic['query']
        total += 1
        match_flag = False

        for k, v in keywords.items():
            must_flag = True
            for item in v['must']:
                if item not in words:
                    must_flag = False
                    break
            if must_flag:
                must_or_flag = False
                if len(v['must_or']) == 0:
                    must_or_flag = True
                else:
                    for item in v['must_or']:
                        if item in words:
                            must_or_flag = True
                            break
            if must_flag and must_or_flag:
                should_flag = False
                if len(v['should']) == 0:
                    should_flag = True
                else:
                    for item in v['should']:
                        if item in words:
                            should_flag = True
                            break
            if must_flag and must_or_flag and should_flag:
                keyword_dict[k] += 1
                match_flag = True
                dic['label_2nd'] = k
                mark = dic["query"] + dic["session_id"]

        if match_flag == False:
            surrender_unmatch_count += 1
            dic['label_2nd'] = "平台体验咨询"

        if match_flag:
            platform_match_count += 1
            
platform_cover_rate = round(platform_match_count / (platform_match_count + platform_unmatch_count), 4) * 100
            
label_frame.append(["平台体验", platform_match_count, platform_cover_rate, 
                    round(platform_match_count / len(PH01_dicts), 4) * 100])

#投保
label_1_dict = {'投保咨询': '投保',
                '产品咨询': '投保',
                '咨询免责条款': '投保',
                '产品对比咨询': '投保',
                '平台产品对比': '投保',
                '社保相关咨询': '投保',
                '咨询产品优惠': '投保',
                '家庭配置需求': '投保',
                '投保材料': '投保',
                '投保条件（地域国籍）': '投保',
                '投保条件（年龄）': '投保',
                '投保条件（职业）': '投保',
                '投保条件（健康情况）': '投保',
                '咨询恶性肿瘤相关': '投保',
                '咨询重大疾病住院津贴保险金': '投保',
                '咨询快速就医': '投保',
                '咨询住院押金垫付': '投保',
                '咨询抗癌特药': '投保',
                '咨询投保优惠': '投保',
                '产品保障期限': '投保',
                '产品保障范围': '投保',
                '赔付次数': '投保',
                '咨询住院津贴': '投保',
                '咨询附加险': '投保',
                '投保流程咨询': '投保',
                '咨询医院范围': '投保',
                '咨询被保人/受益人': '投保',
                '咨询投保人': '投保',
                '投保异常': '投保',
                '产品等待期咨询': '投保',
                '产品犹豫期咨询': '投保',
                '产品免赔额咨询': '投保',
                '咨询产品停售': '投保'}

label1 = list(set(label_1_dict.values()))
label_dict = {}
for k, v in label_1_dict.items():
    if v not in label_dict:
        label_dict[v] = [k]
    else:
        label_dict[v].append(k)

keywords = {

    '投保咨询': {'must': [], 'must_or': ["买", "购买", "投保", "投了", "两份", "几份", "多份"], 'should': []},
    # 具体产品咨询+推荐
    '产品咨询': {'must': [],
             'must_or': ["寿险", "医疗险", "重疾险", "意外险", "消费型", "健康险", "储蓄型", "消费险", "消费性",
                         "哪些", "哪款", "险种", "适合", "推荐", "品种"], 'should': []},
    '咨询免责条款': {'must': [], 'must_or': ["免责", "责任免除", "免责条款"], 'should': []},

    '产品对比咨询': {'must': [], 'must_or': ["香港保险", "香港的保险", "区别", "一样", "哪个好", "分别", "别的", "冲突", "重复"], 'should': []},

    '平台产品对比': {'must': [], 'must_or': ["支付宝", "蚂蚁", "好医保", "众安", "水滴", "悟空", "乐享"], 'should': []},

    '社保相关咨询': {'must': [], 'must_or': ["社保"], 'should': []},

    '咨询产品优惠': {'must': [], 'must_or': ["优惠", "折扣", "打折", "便宜"], 'should': []},

    '家庭配置需求': {'must': [], 'must_or': ["父", "爸", "母", "妈", "公公", "婆婆",
                                       "离婚", "直系", "血缘", "领养",
                                       "子女", "女儿", "儿子", "闺女", "孩", "娃", "宝宝",
                                       "老人", "家人", "家里", "爷", "奶", "外公", "外婆"], 'should': []},

    '投保材料': {'must': [], 'must_or': ["身份证", "证明", "材料", "户口本"], 'should': []},

    '投保条件（地域国籍）': {'must': [], 'must_or': ["外籍", "外国人", "港澳", "澳门", "台湾", "台胞", "留学生", "居住证", "籍"], 'should': []},

    '投保条件（年龄）': {'must': [], 'must_or': ["年龄", "年纪", "岁数"], 'should': []},  # 缺少XX岁

    '投保条件（职业）': {'must': [], 'must_or': ["职业", "工作", "高危",
                                         "司机", "外卖", "修理", "军人", "兵", "警", "货车", "货运", "工", "仓库", "车间",
                                         "船员", "海员", "叉车", "打鱼", "渔民", "出海", "厨师", "高空", "高压电"], 'should': []},

    '投保条件（健康情况）': {'must': [],
                   'must_or': ['血', '胃', '肾', '三高', '重症', '患', '病', '肝', '脂', '症',
                               '轻症', '重症', '身故', '脑', '心',
                               '中毒', '梅毒', '肺', '风湿', '炎', '痛', '病', '牙',
                               '车祸', '抓伤', '咬伤', '骨折', '摔', '扭', '臂', '胳膊',
                               '确诊', '住院', '住过院', '医院', '门诊', '手术'],
                   'should': []},

    '咨询恶性肿瘤相关': {'must': [], 'must_or': ["瘤", "癌", "恶性"], 'should': []},

    '咨询重大疾病住院津贴保险金': {'must': [], 'must_or': ["住院津贴", "津贴"], 'should': []},

    '咨询快速就医': {'must': [],
               'must_or': ['专家导诊', '远程问诊', '专家门诊', '专业陪诊', '快速住院', '专家手术', '专家复诊', '医后随访'], 'should': []},

    '咨询住院押金垫付': {'must': [], 'must_or': ["押金", "垫"], 'should': []},

    '咨询抗癌特药': {'must': [], 'must_or': ["特药", "特效", "靶向", "免疫治疗", "抗癌"], 'should': []},

    '咨询投保优惠': {'must': ["投"], 'must_or': ["优惠", "折扣", "打折", "便宜"], 'should': []},

    '产品保障期限': {'must': [], 'must_or': ["期限", "多久", "终身", "一辈子", "一直保", "多长时间", "保几年"], 'should': []},

    '产品保障范围': {'must': [], 'must_or': ["保障范围", "病种", "哪些病", "哪些疾病", "保不保", ], 'should': []},

    '赔付次数': {'must': [], 'must_or': ["赔付多次", "多次赔付", "赔付一次", "一次赔付", '叠加赔付', '双倍赔付', '一次性'], 'should': []},

    '咨询住院津贴': {'must': [], 'must_or': ["住院津贴", "津贴"], 'should': []},
    '咨询附加险': {'must': [], 'must_or': ["附加险", "特需医疗"], 'should': []},
    '投保流程咨询': {'must': [], 'must_or': ["流程", "怎么", "操作"], 'should': []},
    '咨询医院范围': {'must': [], 'must_or': ["县级", "二级", "三甲", "社康", "医院", "指定医院", "定点医院", "乡镇医院"], 'should': []},
    '咨询被保人/受益人': {'must': [], 'must_or': ["受益人", "被保险人", "被保人"], 'should': []},
    '咨询投保人': {'must': [], 'must_or': ["投保人"], 'should': []},
    '投保异常': {'must': [], 'must_or': ["买不了", "不成功", "失败"], 'should': []},
    '产品等待期咨询': {'must': [], 'must_or': ["等待期"], 'should': []},
    '产品犹豫期咨询': {'must': [], 'must_or': ["犹豫期"], 'should': []},
    '产品免赔额咨询': {'must': [], 'must_or': ["免赔额", "免额"], 'should': []},
    '咨询产品停售': {'must': [], 'must_or': ["停售", "下架", "下线"], 'should': []}
}

keyword_dict = {}
for item in keywords.keys():
    keyword_dict[item] = 0

surrender_match_count = 0
surrender_unmatch_count = 0

for dic in PH01_dicts:
    if dic['label_1st'] == "投保" or dic['label_1st'] == "产品咨询":
        dic['label_1st'] = "投保"
        words = dic['query']
        total += 1
        match_flag = False

        for k, v in keywords.items():
            must_flag = True
            for item in v['must']:
                if item not in words:
                    must_flag = False
                    break
            if must_flag:
                must_or_flag = False
                if len(v['must_or']) == 0:
                    must_or_flag = True
                else:
                    for item in v['must_or']:
                        if item in words:
                            must_or_flag = True
                            break
            if must_flag and must_or_flag:
                should_flag = False
                if len(v['should']) == 0:
                    should_flag = True
                else:
                    for item in v['should']:
                        if item in words:
                            should_flag = True
                            break
            if must_flag and must_or_flag and should_flag:
                keyword_dict[k] += 1
                match_flag = True
                dic['label_2nd'] = k
                mark = dic["query"] + dic["session_id"]

        if match_flag == False:
            surrender_unmatch_count += 1
            dic['label_2nd'] = "投保咨询"

        if match_flag:
            surrender_match_count += 1
            
surrender_cover_rate = round(surrender_match_count / (surrender_match_count + surrender_unmatch_count), 4) * 100
            
label_frame.append(["投保", surrender_match_count, surrender_cover_rate, 
                    round(surrender_match_count / len(PH01_dicts), 4) * 100])

#活动增值服务
label_1_dict = {

    '有哪些活动服务': '活动增值服务',
    '赠送的活动服务是否收费': '活动增值服务',
    '怎么领取查询使用活动服务': '活动增值服务',
    '对活动服务有异议': '活动增值服务'

}

label1 = list(set(label_1_dict.values()))
label_dict = {}
for k, v in label_1_dict.items():
    if v not in label_dict:
        label_dict[v] = [k]
    else:
        label_dict[v].append(k)

keywords = {

    '有哪些活动服务': {'must': [], 'must_or': ["礼品", "礼包", "福利", "优惠", "赠送", "洗牙", "会员", "活动", "享受", "券", "卡券"], 'should': []},

    '赠送的活动服务是否收费': {'must': [], 'must_or': ["交钱", "要钱", "收钱", "收费", "免费"], 'should': []},

    '怎么领取查询使用活动服务': {'must': [], 'must_or': ["哪里", "领", "找", "查", "使用", "兑换", "入口", "指定", "操作", "界面"], 'should': []},

    '对活动服务有异议': {'must': [], 'must_or': ["过期", "到期", "失效", "下线", "下架", "不要", "不需要", "找不到",
                                         "限制", "没收到", "无法使用", "结束", "刷新", "换成", "不好", "没到货"], 'should': []}

}

keyword_dict = {}
for item in keywords.keys():
    keyword_dict[item] = 0
total = 0
activity_match_count = 0
activity_unmatch_count = 0

for dic in PH01_dicts:
    if dic['label_1st'] == "活动增值服务":
        words = dic['query']
        total += 1
        match_flag = False

        for k, v in keywords.items():
            must_flag = True
            for item in v['must']:
                if item not in words:
                    must_flag = False
                    break
            if must_flag:
                must_or_flag = False
                if len(v['must_or']) == 0:
                    must_or_flag = True
                else:
                    for item in v['must_or']:
                        if item in words:
                            must_or_flag = True
                            break
            if must_flag and must_or_flag:
                should_flag = False
                if len(v['should']) == 0:
                    should_flag = True
                else:
                    for item in v['should']:
                        if item in words:
                            should_flag = True
                            break
            if must_flag and must_or_flag and should_flag:
                keyword_dict[k] += 1
                match_flag = True
                dic['label_2nd'] = k
                mark = dic["query"] + dic["session_id"]

        if match_flag == False:
            activity_unmatch_count += 1
            dic['label_2nd'] = "咨询活动服务"

        if match_flag:
            activity_match_count += 1
            
activity_cover_rate = round(activity_match_count / (activity_match_count + activity_unmatch_count), 4) * 100
            
label_frame.append(["活动增值服务", activity_match_count, activity_cover_rate, 
                    round(activity_match_count / len(PH01_dicts), 4) * 100])

            
#二级标签打标结果写入文件
output = open(output_file, 'w', encoding="utf-8")
for row in PH01_dicts:
    tmp = json.dumps(row, ensure_ascii=False)
    output.write(tmp)
    output.write('\n')
output.close()



200000
退保类目二级标签覆盖率为：50.1%
续保类目二级标签覆盖率为：71.08%
理赔类目二级标签覆盖率为：65.45%


In [6]:
PH01_dicts[-1]

{'ws_user_id': '4c2746c6d39c45f0b031a5317a6d6dfc',
 'userid': 'wmLpClCAAAX-9xYCbA7zlv2mARbPeYRA',
 'user_flag': 0,
 'user_name': '万义',
 'session_time': '2021-01-05 14:01:08',
 'create_time': '2021-01-05 14:04:44',
 'direction': 0,
 'message_type': 'text',
 'is_faq': 0,
 'label_1st': '其他',
 'query': '我知道',
 'session_id': '21010596ba6c45e9962b531b6e060f61'}

In [7]:
PH01_dicts[0]

{'ws_user_id': '878da5995af3491f99ba0ff5926365bc',
 'userid': 'wmLpClCAAAjdWvPHboLxS0IWtoV0BkaA',
 'user_flag': 0,
 'user_name': '未来的.路',
 'session_time': '2021-01-19 14:14:45',
 'create_time': '2021-01-19 15:12:27',
 'direction': 0,
 'message_type': 'text',
 'is_faq': 0,
 'label_1st': '其他',
 'query': '[疑问]',
 'session_id': '210119ce6ac44fbe9af931befde5f20b'}

In [8]:
import pandas as pd
from collections import Counter

first_label = [dic["label_1st"] for dic in PH01_dicts]
first_label_set = list(set(first_label))

label_counter = Counter(first_label).most_common(len(first_label_set))

label_frame = []
for i in range(len(label_counter)):
    label = label_counter[i][0]
    count = label_counter[i][1]
    percent = round(count / len(first_label), 4) *100
    label_frame.append([label, count, str(percent) + "%"])

filename = "VOC数据覆盖率逐级统计.xlsx"

df = pd.DataFrame()
df["label_1st"] = [x[0] for x in label_frame]
df["count"] = [x[1] for x in label_frame]
df["percent"] = [x[2] for x in label_frame]

df.to_excel("0304全量数据一级标签占比.xlsx", index=False)

def second_label_count(dic, label, file_name):
    sub_dic = [dic for dic in PH01_dicts if dic["label_1st"] == label]
    if "label_2nd" in sub_dic[0].keys():
        second_label = [dic["label_2nd"] for dic in sub_dic]
        label_counter = Counter(second_label).most_common(len(list(set(second_label))))
        label_frame = []
        for i in range(len(label_counter)):
            label_tmp = label_counter[i][0]
            count_tmp = label_counter[i][1]
            percent_tmp = round(count_tmp / len(sub_dic), 4) *100
            percent_all_tmp = round(count_tmp / len(PH01_dicts), 4) *100
            label_frame.append([label, count_tmp, str(percent_tmp) + "%", str(percent_all_tmp) + "%", label_tmp])
        
        df = pd.DataFrame()
        df["label_1st"] = [x[0] for x in label_frame]
        df["label_2nd"] = [x[4] for x in label_frame]
        df["count"] = [x[1] for x in label_frame]
        df["percent_in_1st_label"] = [x[2] for x in label_frame]
        df["percent"] = [x[3] for x in label_frame]
    
        df.to_excel(file_name + label + ".xlsx", index=False, sheet_name = label)
    else:
        print("no 2nd label yet~")

filename_sub = "VOC数据覆盖率逐级统计_"

for line in first_label_set:
    second_label_count(dic, line, filename_sub)

no 2nd label yet~
no 2nd label yet~
no 2nd label yet~
no 2nd label yet~
